In [209]:
import pandas as pd
import numpy as np

In [210]:
train = pd.read_pickle('Transactions.pkl')
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 876032 entries, 0 to 261675
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   datetime     876032 non-null  object
 1   custid       876032 non-null  int64 
 2   store        876032 non-null  object
 3   product      876032 non-null  int64 
 4   brand        876032 non-null  object
 5   corner       876032 non-null  object
 6   import       876032 non-null  int64 
 7   amount       876032 non-null  int64 
 8   installment  876032 non-null  int64 
dtypes: int64(5), object(4)
memory usage: 66.8+ MB


In [211]:
train.head()

,datetime,custid,store,product,brand,corner,import,amount,installment
0,2000-10-17 18:33:00,16,천호점,4100110001100,비비안스타킹,섬유,0,25000,1
1,2001-01-09 17:43:00,16,신촌점,2139143008000,비오뗌,화장품,0,30000,1
2,2001-01-08 18:50:00,16,천호점,4218410011030,예스비,영캐주얼,0,131000,6
3,2001-03-21 19:13:00,16,천호점,4209563011000,SJ,영캐주얼,0,330000,5
4,2001-04-10 12:00:00,16,천호점,4242470011010,올리브,영캐주얼,0,214200,3


#### object -> datetime

In [212]:
train['datetime'] = pd.to_datetime(train['datetime'])

In [213]:
train['month'] = train['datetime'].dt.month
train['hour'] = train['datetime'].dt.hour
train['day'] = train['datetime'].dt.day_name().str[:3]

In [214]:
train['weekend'] = ["주말" if value in ['Sun', 'Sat'] else "평일" for value in train['day']]

### Make Features

In [216]:
id_g = train.groupby('custid')

#### 1. 구매횟수

In [217]:
cp1 = id_g.size().reset_index(name = '구매횟수')
cp1.head()

,custid,구매횟수
0,16,34
1,29,19
2,33,56
3,34,23
4,50,36


#### 2. 구매 브랜드 수

In [218]:
cp2 = id_g.brand.nunique().reset_index(name = '구매브랜드수')
cp2.head()

,custid,구매브랜드수
0,16,23
1,29,9
2,33,27
3,34,16
4,50,27


#### 3. 구매 코너 수

In [219]:
cp3 = id_g.corner.nunique().reset_index(name = '구매코너수')
cp3.head()

,custid,구매코너수
0,16,10
1,29,6
2,33,12
3,34,10
4,50,13


#### 4. 주말 방문 비율

In [220]:
cp4 = pd.crosstab(train.custid, train['weekend']).reset_index()
cp4.index.name = None
cp4.columns.name = None

In [221]:
cp4['주말비율'] = cp4.주말 / cp1.구매횟수

In [222]:
cp4.drop(['주말', '평일'], axis = 1, inplace = True)

In [223]:
cp4.head()

,custid,주말비율
0,16,0.147059
1,29,0.894737
2,33,0.642857
3,34,0.826087
4,50,0.500000


#### 5. 수입상품비율

In [224]:
cp5 = pd.crosstab(train.custid, train['import']).reset_index()
cp5.index.name = None
cp5.columns.name = None

In [225]:
cp5['수입상품비율'] = cp5.iloc[:, 2] / cp1.구매횟수

In [226]:
cp5 = cp5.iloc[:, [0, -1]]

In [227]:
cp5.head()

,custid,수입상품비율
0,16,0.176471
1,29,0.052632
2,33,0.142857
3,34,0.086957
4,50,0.000000


#### 6. 평균 구매 시간

In [228]:
cp6 = id_g.hour.mean().reset_index(name = '평균구매시간')
cp6.head()

,custid,평균구매시간
0,16,15.911765
1,29,16.473684
2,33,15.446429
3,34,16.391304
4,50,16.388889


#### 7. 평균 할부 개월 수

In [229]:
cp7 = id_g.installment.mean().reset_index(name = '평균할부개월수')
cp7.head()

,custid,평균할부개월수
0,16,3.470588
1,29,2.157895
2,33,2.107143
3,34,2.565217
4,50,2.416667


#### 8. 구매 주기

In [230]:
cp8 = id_g.datetime.agg([('최근방문', 'max'), ('첫방문', 'min')]).reset_index()

In [231]:
cp8['경과일수'] = (cp8.최근방문 - cp8.첫방문).dt.days

In [232]:
cp8['평균방문주기'] = cp8.경과일수 / cp1.구매횟수

In [233]:
cp8 = cp8.iloc[:, [0,3,4]]

In [234]:
cp8.head()

,custid,경과일수,평균방문주기
0,16,174,5.117647
1,29,169,8.894737
2,33,126,2.250000
3,34,139,6.043478
4,50,331,9.194444


#### 9. 평균 구매 금액

In [235]:
cp9 = id_g.amount.mean().reset_index(name = '평균구매금액')
cp9.head()

,custid,평균구매금액
0,16,83148.058824
1,29,50891.052632
2,33,90706.589286
3,34,72302.173913
4,50,99450.000000


#### 10. 할부 구매 건수

In [236]:
ins_cnt = pd.crosstab(train['custid'], train['installment']).reset_index()
ins_cnt.index.name = None
ins_cnt.columns.name = None

In [237]:
ins_cnt['할부구매비율'] = ins_cnt.iloc[:, 1] / ins_cnt.iloc[:, 1:].sum(axis = 1)

In [238]:
cp10 = ins_cnt.iloc[:, [0, -1]]

In [277]:
cp10.head()

,custid,할부구매비율
0,16,0.264706
1,29,0.421053
2,33,0.517857
3,34,0.217391
4,50,0.333333


***

### Make Data Set

In [239]:
df = pd.merge(cp1, cp2, how = 'left')
df = pd.merge(df, cp3, how = 'left')
df = pd.merge(df, cp4, how = 'left')
df = pd.merge(df, cp5, how = 'left')
df = pd.merge(df, cp6, how = 'left')
df = pd.merge(df, cp7, how = 'left')
df = pd.merge(df, cp8, how = 'left')
df = pd.merge(df, cp9, how = 'left')
df = pd.merge(df, cp10, how = 'left')

In [256]:
cs = pd.read_pickle('Customers.pkl')
cs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18341 entries, 0 to 5501
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   custid       18341 non-null  int64  
 1   gender       18341 non-null  int64  
 2   birth        18341 non-null  object 
 3   house_type1  18341 non-null  object 
 4   house_type2  18341 non-null  int64  
 5   hobby        18341 non-null  int64  
 6   marriage     12839 non-null  float64
dtypes: float64(1), int64(4), object(2)
memory usage: 1.1+ MB


In [241]:
cs['age'] = 2021 - cs['birth'].str[:4].astype(int)

In [242]:
cs = cs.drop('birth', axis = 1)

In [243]:
df = pd.merge(df, cs, how = 'left')

***
### Split Train & Test

In [260]:
train = df[df.marriage.notnull()]

In [261]:
test = df[df.marriage.isnull()]

In [262]:
train = pd.get_dummies(train)

In [263]:
test = pd.get_dummies(test)

In [267]:
train['house_type1_$null$'] = 0
train['house_type1_H'] = 0

***
### Modeling

In [198]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

In [272]:
X = train.drop(['custid', 'marriage'], axis = 1)
y = train['marriage']

- 모델링에 불필요한 컬럼인 custid와 종속변수인 marriage를 제외한 학습용 데이터를 X 변수에 지정한다.
- 예측 대상인 marriage 변수를 y 변수에 지정한다.

In [273]:
target = test.drop(['custid', 'marriage'], axis = 1)

- 마찬가지로 모델링에 불필요한 컬럼인 custid와 종속변수인 marriage를 제외한 테스트 데이터를 target 변수에 지정한다.

In [274]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

- X의 80%를 학습용으로 사용하고 20%를 검증용으로 사용한다. 이 때 분류이기 때문에 클래스 비율을 맞추기 위해 stratify = y로 지정해준다.

In [283]:
skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

- 일반적인 KFold와 달리 분류의 경우 StratifiedKFold(층화추출)을 교차검증에 사용한다. 데이터를 총 5개의 FOLD로 나누기 위해 세팅한다.

***
### HyperParameter Tuning
-  Only Using X/y train

In [ ]:
params = {
    'n_estimators': [10, 300, 500, 750, 1000],
    'max_depth': [3, 4, 5, 6, 7],
    'max_features': [3, 4, 5, 6, 7]
}

- 탐색할 파라미터 조합을 의미한다. RandomForest의 파라미터는 굉장히 많으며 대표적인 3개의 파라미터에 대해 탐색할 값을 지정한다.

In [ ]:
rf = RandomForestClassifier(random_state = 42, n_jobs = -1)

- 탐색할 모델을 지정한다.

In [ ]:
rand_search = RandomizedSearchCV(rf, params, scoring = 'neg_log_loss', cv = skf)

- 랜덤하게 파라미터 조합을 찾도록 RandomizedSearchCV를 이용한다. 이 때 평가지표인 log_loss를 사용한다. 
- 앞에 neg_가 붙는 이유는 일반적으로 파라미터 튜닝은 값이 크게 나오도록 모델을 탐색한다. log_loss의 경우 값이 낮을수록 좋은 성능을 의미하기 때문에 앞에 neg가 붙는다.
- log loss가 각각 0.3, 0.7인 경우에 파라미터튜닝 결과는 더 큰 값인 0.7을 성능으로 제시한다. 하지만 log_loss는 값이 작은 것이 좋은 성능이다. 따라서 앞에 음수를 붙이면 -0.3, -0.7이기 때문에 더 큰 값인 -0.3이라는 결과를 반환하기 위해 파라미터를 튜닝한다.
- cv는 cross_validation으로 앞서 지정한 StratifiedKFold를 사용하겠다는 의미이다.

In [297]:
rand_search.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
                   param_distributions={'max_depth': [3, 4, 5, 6, 7],
                                        'max_features': [3, 4, 5, 6, 7],
                                        'n_estimators': [10, 300, 500, 750,
                                                         1000]},
                   scoring='neg_log_loss')

- 실제로 학습데이터로 파라미터 조합을 탐색한다.

In [298]:
best_params = rand_search.best_params_

In [299]:
best_params

{'n_estimators': 10, 'max_features': 6, 'max_depth': 7}

- best_params_라는 메서드에 최적의 파라미터 조합이 저장되어 있다.

In [300]:
pred_proba = rand_search.predict_proba(X_val)
score = log_loss(y_val, pred_proba)
print(f'Log Loss in Cross Validation : {score}')

Log Loss in Cross Validation:  0.3436128446955919


- 최적의 파라미터 조합이 세팅되어 있는 모델로 검증용 데이터를 활용하여 예측값을 도출한다.
- 검증용 y 데이터와 log_loss를 계산한다.

***
### Experiments

In [313]:
rf1 = RandomForestClassifier(n_estimators = 100, max_features = 6, max_depth = 5, random_state = 42, n_jobs =- 1)
rf2 = RandomForestClassifier(n_estimators = 500, max_features = 7, max_depth = 4, random_state = 42, n_jobs =- 1)
rf3 = RandomForestClassifier(n_estimators = 1000, max_features = 4, max_depth = 3, random_state = 42, n_jobs =- 1)

pred1 = rf1.fit(X, y).predict_proba(X_val)
pred2 = rf2.fit(X, y).predict_proba(X_val)
pred3 = rf3.fit(X, y).predict_proba(X_val)

print(f'Expected Log Loss1: ', log_loss(y_val, pred1))
print(f'Expected Log Loss2: ', log_loss(y_val, pred2))
print(f'Expected Log Loss3: ', log_loss(y_val, pred3))

Expected Log Loss1:  0.3274405609868347
Expected Log Loss2:  0.33481299255824637
Expected Log Loss3:  0.3560934202495027


***
### Final Decision & Prediction

- RandomizedSearchCV Using All Data(not X/y train)

In [ ]:
params = {
    'n_estimators': [10, 300, 500, 750, 1000],
    'max_depth': [3, 4, 5, 6, 7],
    'max_features': [3, 4, 5, 6, 7]
}

In [ ]:
rf = RandomForestClassifier(random_state = 42, n_jobs = -1)

In [ ]:
rand_search = RandomizedSearchCV(rf, params, scoring = 'neg_log_loss', cv = skf)

In [314]:
rand_search.fit(X, y) # Use All Data for Train

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
                   param_distributions={'max_depth': [3, 4, 5, 6, 7],
                                        'max_features': [3, 4, 5, 6, 7],
                                        'n_estimators': [10, 300, 500, 750,
                                                         1000]},
                   scoring='neg_log_loss')

- 최종 예측을 할 때는 전체 데이터를 cross_validation으로 사용하며 모든 데이터를 활용한다.

In [315]:
final_params = rand_search.best_params_

- 최적의 파라미터 조합이 세팅되어 있다.

In [ ]:
final_model = RandomForestClassifier(**final_params, random_state = 42, n_jobs =- 1)

- **params를 인자로 지정하면 우리가 찾았던 파라미터 조합이 알맞게 자동으로 세팅된다.

In [317]:
final_model.fit(X,y)

RandomForestClassifier(max_depth=7, max_features=5, n_estimators=1000,
                       n_jobs=-1, random_state=42)

In [318]:
final_pred = final_model.predict_proba(target)

- 최종적으로 테스트 데이터로 예측 값을 도출한다.

In [319]:
submission_jeonchihong1 = pd.DataFrame(final_pred, index = test.custid)

- 데이터 프레임에 값은 결혼여부 0, 1에 대한 확률값이 들어가고 인덱스는 테스트 데이터의 custid로 지정한다.

In [320]:
submission_jeonchihong1.head()

,0,1
custid,,
80,0.598373,0.401627
81,0.624771,0.375229
93,0.619603,0.380397
117,0.661774,0.338226
129,0.794925,0.205075


In [ ]:
submission_jeonchihong1.to_csv('data/Submission_JeonChihong.csv', index = False)

- 위의 데이터 프레임을 최종 정답 제출용 파일로 저장한다.